In [10]:
import numpy as np
import pandas as pd

In [11]:
df = pd.read_csv("Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [12]:
#normalisation
df.fillna(df.mean(),inplace=True)
df

C:\Users\bt3222\AppData\Local\Temp\ipykernel_13880\2217261690.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(),inplace=True)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [13]:
from math import log2
def calculate_entropy(data):
    class_counts=data.value_counts()
    total_samples=len(data)
    entropy =-sum((count/total_samples)*log2(count/total_samples)for count in class_counts)
    return entropy
def information_gain(data,attribute,target):
    total_entropy=calculate_entropy(data[target])
    values=data[attribute].unique()
    n=len(data[attribute])
    weighted_entropy=0

    for value in values:
        subset= data[data[attribute]==value]
        set_size=len(subset)
        weighted_entropy+=(set_size/n)*(calculate_entropy(subset[target]))
    gain=total_entropy-weighted_entropy

    return gain



In [14]:
from sklearn.model_selection import train_test_split



In [15]:
from math import log2
import pandas as pd
import numpy as np

# Function to calculate entropy
def calculate_entropy(data):

    class_counts = data.value_counts()
    total_samples = len(data)
    entropy = -sum((count / total_samples) * log2(count / total_samples) for count in class_counts if count > 0)
    return entropy

# Function to calculate information gain
def information_gain(data, attribute, target):

    total_entropy = calculate_entropy(data[target])
    values = data[attribute].unique()
    total_samples = len(data)
    weighted_entropy = 0

    for value in values:
        subset = data[data[attribute] == value]
        subset_size = len(subset)
        weighted_entropy += (subset_size / total_samples) * calculate_entropy(subset[target])
    
    gain = total_entropy - weighted_entropy
    return gain

# Function to find the best split
def best_split(data, target):

    attributes = [col for col in data.columns if col != target]
    best_gain = -1
    best_attribute = None

    for attribute in attributes:
        gain = information_gain(data, attribute, target)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    
    return best_attribute, best_gain

# Function to build the decision tree
def build_tree(data, target, max_depth=None, depth=0):
 
    if max_depth is not None and depth >= max_depth:
        return data[target].mode()[0]
    if len(data[target].unique()) == 1:
        return data[target].iloc[0]
    if len(data.columns) == 1:
        return data[target].mode()[0]
    
    best_attribute, _ = best_split(data, target)
    if best_attribute is None:
        return data[target].mode()[0]

    tree = {best_attribute: {}}
    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value].drop(columns=[best_attribute])
        subtree = build_tree(subset, target, max_depth, depth + 1)
        tree[best_attribute][value] = subtree
    
    return tree

def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree
    
    attribute = next(iter(tree))
    value = sample[attribute]
    subtree = tree[attribute].get(value, None)

    if subtree is None:
        return None
    return predict(subtree, sample)

def call():
    from sklearn.model_selection import train_test_split

    feature_cols= ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']
    X = df[feature_cols]
    Y=df['Species']

    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
    decision_tree = build_tree(train_data, target="target", max_depth=3)
    predictions = test_data.apply(lambda row: predict(decision_tree, row), axis=1)
    accuracy = np.mean(predictions == test_data["target"])
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("Decision Tree:", decision_tree)


In [16]:
call()

KeyError: 'target'